In [1]:
import cv2
import numpy as np

#1
roi = None
drag_start = None
mouse_status = 0
tracking_start = False
# 4개 변수 초기화, 마우스 이벤트에 사용

# mouse event 함수 정의
def onMouse(event, x, y, flags, param=None):
    global roi
    global drag_start
    global mouse_status
    global tracking_start
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drag_start = (x, y) # 마우스 드래그 포인트
        mouse_status = 1  # 1은 roi를 설정하기 시작했다는 의미
        tracking_start = False # 아직 트랙킹 안 한 상태
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags == cv2.EVENT_FLAG_LBUTTON:
            xmin = min(x, drag_start[0])
            ymin = min(y, drag_start[1])
            xmax = max(x, drag_start[0])
            ymax = max(y, drag_start[1])
            roi = (xmin, ymin, xmax, ymax)
            mouse_status = 2 # dragging
            # 마우스 시작포인트, 끝 포인트 즉 roi 영역 저장, 드래그 하는 중이라는 의미
            
    elif event == cv2.EVENT_LBUTTONUP:
        mouse_status = 3 # complete 마우스에서 손을 떼었다라는 의미

In [2]:
#2
cv2.namedWindow('tracking') # tracking 이라는 윈도우 생성
cv2.setMouseCallback('tracking', onMouse) # onMous에 등록

path = '/Users/werther/image/'
cap = cv2.VideoCapture(path + 'lucas3.mp4') # 새로운 영상 읽기
if (not cap.isOpened()):
    print('Error opening video')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), 
                 int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
roi_mask = np.zeros((height, width), dtype=np.uint8) 
# 동영상과 같은 사이즈의 roi 마스크 생성. black

params = dict(maxCorners=16, qualityLevel=0.1, minDistance=1, blockSize=5)
# 밑에 goodFeaturesToTrack에 사용
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
# 계속 돌아가지 않게 종료 조건 설정 시 사용, 10번
params2 = dict(winSize=(5,5), maxLevel=3, criteria=term_crit)
# 루카스 카나데 opticalflow 함수에 넣는 파라미터

#3
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t += 1
    print('t =', t)
    imgC = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    imgC = cv2.GaussianBlur(imgC, (5,5), 0.0)
    # 프레임 가져와서 gray는 흑백으로 변경 후 가우시안 블러 적용
    
    #3-1
    if mouse_status==2: # 마우스 드래그 중
        x1, y1, x2, y2 = roi
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,0,255), 2)
    
    #3-2
    if mouse_status==3: # 마우스 손 뗀 상태, roi 설정 완료
        print('initialize...')
        mouse_status = 0
        x1, y1, x2, y2 = roi       # 좌표에 따른 roi 설정
        roi_mask[:, :] = 0         # 원래 블랙 
        roi_mask[y1:y2, x1:x2] = 1 # roi 영역만 1로 셋팅
        p1 = cv2.goodFeaturesToTrack(imgC, mask=roi_mask, **params) # 피처를 찾기 위해 좋은 트랙킹을 찾는다, 최종 피처가 남는다
        if len(p1)>=4:             # 피처가 4개 이상이면
            p1 = cv2.cornerSubPix(imgC, p1, (5,5), (-1,-1), term_crit) # 코너를 찾는다
            rect = cv2.minAreaRect(p1) # 코너가 이루는 최소 사각형 구간
            box_pts = cv2.boxPoints(rect).reshape(-1,1,2) # 포인트로 변경 저장
            tracking_start = True # 트랙킹 시작
            
    #3-3
    if tracking_start: # 트랙킹 시작되었다면,
        p2, st, err = cv2.calcOpticalFlowPyrLK(imgP, imgC, p1, None, **params2) #옵티컬플로우 피라미드 LK
        p1r, st, err = cv2.calcOpticalFlowPyrLK(imgC, imgP, p2, None, **params2)
        # imgC는 가우시안 블러한 현재 프레임, imgP는 현재 프레임을 카피한 것. 반복문이므로 다음 루프 때 이전 루프의 imgC와 동일
        # 즉 imgC는 current frame, imgP는 previous frame
        # p2에 이전 프레임 ~ 현재 프레임 optical flow 저장, p1r에 현재 프레임 ~ 이전 프레임 optical flow 저장
        # 이유는 두 개의 차이가 적어야 optical flow에 문제가 없는 것
        d = abs(p1-p1r).reshape(-1,2).max(-1)
        # 그래서 두 차이를 계산
        stat = d < 1.0 # 1.0 is distance threshold, 차이 기준 1
        good_p2 = p2[stat==1].copy() # p2는 optical flow
        good_p1 = p1[stat==1].copy() # p1은 good feature
        print(height, width)
        print(good_p2)
        for x, y in good_p2.reshape(-1, 2): # p2에 저장되어 있는 것들을 원으로 표시
            cv2.circle(frame, (int(x), int(y)), 3, (0,0,255), -1)
        
        if len(good_p2)<4: # 4보다 적으면 넘어감
            continue
        
        H, mask = cv2.findHomography(good_p1, good_p2, cv2.RANSAC, 3.0) # p1, p2로 네모 찾음 (매칭)
        box_pts = cv2.perspectiveTransform(box_pts, H) # 찾고자 하는 오브젝트의 바운딩 박스 생성
        cv2.polylines(frame, [np.int32(box_pts)], True, (255,0,0), 2) # 바운딩 박스 그림
        p1 = good_p2.reshape(-1,1,2)
        
    #3-4
    cv2.imshow('tracking', frame)
    imgP = imgC.copy()
    key = cv2.waitKey(25)
    if key == 27:
        break

if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

t = 1
t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9
t = 10
t = 11
t = 12
t = 13
t = 14
t = 15
t = 16
t = 17
t = 18
t = 19
t = 20
t = 21
t = 22
t = 23
t = 24
t = 25
t = 26
t = 27
t = 28
t = 29
t = 30
t = 31
t = 32
t = 33
t = 34
t = 35
t = 36
t = 37
t = 38
t = 39
t = 40
t = 41
t = 42
t = 43
t = 44
t = 45
t = 46
t = 47
t = 48
t = 49
t = 50
t = 51
t = 52
t = 53
t = 54
t = 55
t = 56
t = 57
t = 58
t = 59
t = 60
t = 61
t = 62
t = 63
t = 64
t = 65
t = 66
t = 67
t = 68
t = 69
t = 70
t = 71
t = 72
t = 73
t = 74
t = 75
t = 76
t = 77
t = 78
t = 79
t = 80
t = 81
t = 82
t = 83
t = 84
t = 85
t = 86
t = 87
t = 88
t = 89
t = 90
t = 91
t = 92
t = 93
t = 94
t = 95
t = 96
t = 97
t = 98
t = 99
t = 100
t = 101
t = 102
t = 103
t = 104
t = 105
t = 106
t = 107
t = 108
t = 109
t = 110
t = 111
t = 112
t = 113
t = 114
t = 115
t = 116
t = 117
t = 118
t = 119
t = 120
t = 121
t = 122
t = 123
t = 124
t = 125
t = 126
t = 127
t = 128
t = 129
t = 130
t = 131
t = 132
t = 133
t = 134
t = 135
t = 136
t = 137
t = 138
t = 

TypeError: object of type 'NoneType' has no len()